# histogram with multiple data

In [ ]:
%gui qt

## Load data from a tiff

In [ ]:
import fabio
import numpy
pathImg1="data/IMG_20161103_114423.tiff"
pathImg2="data/IMG_20161103_114433.tiff"
image1 = numpy.max(fabio.open(pathImg1).data, axis=-1)
image2 = numpy.max(fabio.open(pathImg2).data, axis=-1)

## create one histogram for the color intensity
- one for the image1
- one for the image2
- one for the image1 + image 2

![color_distribution](img/plot_advanced_color_distribution.png)

In [ ]:
from silx.math.histogram import Histogramnd, HistogramndLut
import numpy
nb_bins=256
histogram1=Histogramnd(image1.astype(numpy.float32).flatten(), 
                       n_bins=nb_bins, 
                       histo_range=[0,nb_bins])
firstImgHisto=histogram1.histo.copy()

histogram1.accumulate(image2.astype(numpy.float32).flatten())
firstAndSecondImgHisto=histogram1.histo

histogram2=Histogramnd(image2.astype(numpy.float32).flatten(), 
                       n_bins=nb_bins, 
                       histo_range=[0,nb_bins])
secondImgHisto=histogram2.histo

## plot it

In [ ]:
from silx.gui.plot import Plot1D
plot=Plot1D()
plot.addCurve(range(nb_bins), firstImgHisto, 'firstImage', color='black')
plot.addCurve(range(nb_bins), secondImgHisto, 'secondImage', color='blue')
plot.addCurve(range(nb_bins), firstAndSecondImgHisto, 
              'firstAndSecondImage', color='red')
plot.show()

# histogram for weights weights

- simple example : use the pixel values as weights on multiple data

## using histogramnd

### Load data

In [ ]:
# from h5
dataPath="data/Pilatus1M.hdf5"
import h5py
import numpy
#select the cube values:
f=h5py.File(dataPath)
data=f['data']
data = numpy.array(data, order='C', dtype='float64')

In [ ]:
import numpy
def computeradius(data, xcenter, ycenter):
    
    # do the azimutal integration
    xcenter=180
    ycenter=260
    y, x=numpy.ogrid[:data.shape[0], :data.shape[1]]
    r=numpy.sqrt((x-xcenter)**2+(y-ycenter)**2)
    return r

In [ ]:
radii=computeradius(data, xcenter=180, ycenter=260)
nb_bins=int(numpy.ceil(radii.max()))
histo_range=[0,nb_bins]

In [ ]:
def computeHisto_weigh():
    dataset=[]
    for i in range(10):
        dataset.append(data)

    histogram=None
    for d in dataset:
        if histogram is None:
            histogram=Histogramnd(radii.ravel(), 
                                  n_bins=nb_bins, 
                                  histo_range=histo_range, 
                                  weights=d.ravel())
        else:
            histogram.accumulate(radii.ravel(), weights=d.ravel())
            
    return histogram

In [ ]:
h1=computeHisto_weigh()

In [ ]:
p=Plot1D()
binscenter=(h1.edges[0][1:] + h1.edges[0][0:-1]) / 2.0
p.addCurve(x=binscenter, y=h1.weighted_histo, color='red', name='h1')
p.setActiveCurve(None)
p.show()

## using histogramnd_lut

- Histramnd_lut create a look up table to set the association once for all then use it during the accumulate function

In [ ]:
def computeHistoLut_weigh():
    dataset=[]
    for i in range(10):
        dataset.append(data)
        
    histogram=HistogramndLut(radii.ravel(), 
                             n_bins=nb_bins, 
                             histo_range=histo_range)
    for d in dataset:
        histogram.accumulate(d.ravel())
        
    return histogram

In [ ]:
h2=computeHistoLut_weigh()

In [ ]:
p2=Plot1D()
l=h2.weighted_histo()
p2.addCurve(x=binscenter, y=h2.weighted_histo(), color='blue', name='h2')
p2.setActiveCurve(None)
p2.show()

## Then test it with timeit

In [ ]:
%timeit computeHisto_weigh()

In [ ]:
%timeit computeHistoLut_weigh()

In [ ]:
numpy.array_equal(h2.weighted_histo(), h1.weighted_histo)

In [ ]:
numpy.setdiff1d(h2.weighted_histo(), h1.weighted_histo)